In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Pais;
DROP TABLE IF EXISTS Grupo;
DROP TABLE IF EXISTS Infeccoes;


CREATE TABLE Pais (
  nome VARCHAR(60),           
  sigla VARCHAR(3) NOT NULL,
  regiao VARCHAR(30),
  classificacaoRenda VARCHAR(30),
  Gini INTEGER,
  PIBperCapita DOUBLE,
  IDH INTEGER,
  populacao INTEGER,
  PRIMARY KEY(sigla)
) AS SELECT
    country,
    label,
    region,
    wb_class,
    gini,
    gdbPPPperCapita,
    hdi,
    populationTotal
FROM CSVREAD('../../data/processed/countries.csv');

CREATE TABLE Grupo (
  pais VARCHAR(60), 
  id INTEGER,
  PRIMARY KEY(pais)
) AS SELECT
    name, 
    giw
FROM CSVREAD('../../saida/paisXclassificacao.csv');

CREATE TABLE Infeccoes (
  regiao VARCHAR(10),
  ano INTEGER,
  qtde FLOAT
) AS SELECT
    region,
    year,
    nominal
FROM CSVREAD('../../data/processed/infections.csv')


In [3]:
CREATE VIEW GrupoXinfeccao AS
SELECT G.id id, count(*) number_of_countries, 
       AVG(P.idh) idh, AVG(P.gini) gini,
       sum(I.qtde) qtde_infeccoes
FROM Pais AS P, Grupo AS G, Infeccoes AS I
WHERE P.sigla = I.regiao AND P.nome = G.pais AND I.ano=2014
GROUP BY G.id
HAVING number_of_countries > 1
ORDER BY idh, gini desc;

CALL CSVWRITE('../../saida/GrupoXinfeccao.csv', 'SELECT * FROM GrupoXinfeccao');

9